## Reminders of syntax with the facile library

<div class="alert alert-success">
    <b>Hint</b>: Constraints may be summed: the result is the number of True values in the set of constraints.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

Example: Find an array of length $n$ taking values in $[0, n-1]$ so that the $i$-th value is equal to the number of $i$ inside the array.

In [1]:
import facile
n = 10
array = [facile.variable(range(n)) for i in range(n)]
for i in range(n):
    facile.constraint(sum([x == i for x in array]) == array[i])

if facile.solve(array):
    print(f"indices: {list(range(n))}")
    print(f"array:   {[v.value() for v in array]}")

indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
array:   [6, 2, 1, 0, 0, 0, 1, 0, 0, 0]


/Users/z.wu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  import sys


<div class="alert alert-success">
    <b>Hint</b>: You may use the & and | operators for the logical $\land$ and $\lor$.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

In [2]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve([a, b])
a.value(), b.value()

(1, 0)

## Picross

Picross is a popular game of a similar nature to Sudoku.  
It consists of a grid of square dimension $n \times n$ and of a list of integers for each row and each column.

![](./picross_duck.png)

We can use the following notations:

- $b_{i,j} \in \{0, 1\}$ represents a bit, indicating a column or a row where $i, j \in \{1, ..., n\}$ represents a column or row index;
- $m_i \in \{1, ..., \lceil n/2 \rceil\}$ represents the size of the list associated to row/column $i$;
- $a_{i,k} \in \{1, ..., n\}$ is the $k$-th integer of the list associated to row/column $i$.

The goal is to darken some cells of the grid, such that in each row/column $i$ the number of successive black cells form the list $\{a_{i,1}, ..., a_{i,m_i}\}$.

<div class="alert alert-warning">
    <b>Paper submission</b> (half of the grade):<br/>
Before heading to the code, you shall answer the following questions.<br/>
    Please <b>handwrite</b> your answers on a separate sheet of paper. You will submit a pdf scan of this document together with your notebook.  
(You may take a picture with your phone if need be but the pdf option is preferred.)
</div>

1. There are two ways to model the problem:
    - binary variables (1) associated to the color of each cell, or
    - integer variables (2) associated to the bottom/left coordinate of each black cluster.  
  For **both** ways, specify the decision variables and their associated domain.

2. Write the constraint stating that on each line and column, two blocks do not overlap.  
Choose **one** of the two ways of modelling the problem (i.e. decision variables from (1) or from (2)) to express this constraint.

3. Write the constraint that puts a limit to the number of black cells on each line/column.  
Choose **one** of the two ways of modelling the problem (i.e. decision variables from (1) or from (2)) to express this constraint.

4. Write the constraint that links decision variables from (1) to decision variables from (2), stating the conditions under which a cell must be coloured in black.

<div class="alert alert-danger">
    Before you start coding, <b>please take time to understand</b> what happened: we stated two different ways to model the same problem (question 1.). Both are convenient for stating one part of the problem, but cannot state the whole set of constraints (question 2. and 3.).<br/>One constraint (question 4.) helps linking both decision variables.
</div>

You are provided with a set of problems in the Python dictionary `picross`. Your code will be evaluated on all these problems, but you should start with small problems like `moon`, `star` and `cat`. The most time-consuming should be `house` so do not try it before your code is stable.

Have a look at variables `lines` and `columns`, since you will have to feed this data into your problem. From now, it should be (almost) direct to code the constraints you have written on paper into Python.

<div class="alert alert-warning">
    Finally, <b>do not waste too much time on the output</b>.  
    You will <b>not</b> get extra points with a fancy matplotlib plot.  
    A text output, with stars (<code>*</code>) or black characters (<code>■</code>) like the following ones is enough:
</div>

```
         ***                 ■■■    
        *****               ■■■■■   
       **** ***            ■■■■ ■■■ 
       *******             ■■■■■■■  
        *****               ■■■■■   
         ***                 ■■■    
        *****               ■■■■■   
*     ********      ■     ■■■■■■■■  
***  ***   ***      ■■■  ■■■   ■■■  
******* *** **      ■■■■■■■ ■■■ ■■  
 ***** **** **       ■■■■■ ■■■■ ■■  
 ********  **        ■■■■■■■■  ■■   
  **********          ■■■■■■■■■■    
    ** ***              ■■ ■■■      
      ******              ■■■■■■    

```

In [24]:
from picross import picross

# Choose among 'moon', 'star', 'cat', 'horse', 'house' and 'duck'
lines, columns = picross['duck']

print(lines)
print(columns)

n_l, n_c = len(lines), len(columns)

(n_l, n_c)

[[3], [5], [4, 3], [7], [5], [3], [5], [1, 8], [3, 3, 3], [7, 3, 2], [5, 4, 2], [8, 2], [10], [2, 3], [6]]
[[3], [4], [5], [4], [5], [6], [3, 2, 1], [2, 2, 5], [4, 2, 6], [8, 2, 3], [8, 2, 1, 1], [2, 6, 2, 1], [4, 6], [2, 4], [1]]


(15, 15)

In [25]:
m_l = [len(lines[i]) for i in range(n_l)] # 每一行的元素数目
m_c = [len(columns[j]) for j in range(n_c)] # 每一列的元素数目

from facile import *
bl = [array([variable([0,1]) for j in range(n_c)]) for i in range(n_l)]
bc = [array([variable([0,1]) for i in range(n_l)]) for j in range(n_c)]
l = [array([variable(range(n_l)) for k in range(m_l[i])]) for i in range(n_l)]
c = [array([variable(range(n_c)) for k in range(m_c[j])]) for j in range(n_c)]

for i in range(n_l):
    constraint(sum(bl[i])==sum(lines[i]))
for j in range(n_c):
    constraint(sum(bc[j])==sum(columns[j]))
for i in range(n_l):
    for j in range(n_c):
        constraint(bl[i][j] == bc[j][i])
        
for i in range(n_l):
    for k in range(m_l[i]-1):
        constraint(l[i][k+1] - l[i][k] > lines[i][k])
for j in range(n_c):
    for k in range(m_c[j]-1):
        constraint(c[j][k+1] - c[j][k] > columns[j][k])
        
for i in range(n_l):
    for k in range(m_l[i]):
        constraint(sum(bl[i][l[i][k]+p] for p in range(lines[i][k]))==lines[i][k])   
for j in range(n_c):
    for k in range(m_c[j]):
        constraint(sum(bc[j][c[j][k]+p] for p in range(columns[j][k]))==columns[j][k])

white = ' '
black = "*"  
solver = solve([bc_ij for bc_i in bc for bc_ij in bc_i]+[bl_ij for bl_i in bl for bl_ij in bl_i]+[l_ik for l_i in l for l_ik in l_i]+[c_ik for c_i in c for c_ik in c_i])   

if solver:
    for i in range(n_l):
        str = ''
        for j in range(n_c):
            if bl[i][j].value()==0:
                str += white
            else:
                str += black
        print(str)
print(solver)
print("The variable l is: ")
for i in range(n_l):
    print(l[i].value())
print("The variable c is: ")    
for j in range(n_c):
    print(c[j].value())

         ***   
        *****  
       **** ***
       ******* 
        *****  
         ***   
        *****  
*     ******** 
***  ***   *** 
******* *** ** 
 ***** **** ** 
 ********  **  
  **********   
    ** ***     
      ******   
Current solution              : [0, 0, 0, 0, 0, 0, ...]
Resolution status             : True
Resolution time               : 0.97s

The variable l is: 
[9]
[8]
[7, 12]
[7]
[8]
[9]
[8]
[0, 6]
[0, 5, 11]
[0, 8, 12]
[1, 7, 12]
[1, 11]
[2]
[4, 7]
[6]
The variable c is: 
[7]
[8]
[8]
[9]
[9]
[8]
[7, 11, 14]
[2, 7, 10]
[1, 6, 9]
[0, 9, 12]
[0, 9, 12, 14]
[0, 3, 11, 14]
[1, 6]
[2, 7]
[2]
